In [1]:
from main import *
import numpy as np
import sys

In [2]:
def genpairs(n, m, N):
    L=set()
    i, j = np.random.randint(0, n), np.random.randint(0, m)
    L.add((i,j))
    while len(L) < N:
        while (i, j) in L:
            i, j = np.random.randint(0, n), np.random.randint(0,m)
        L.add((i,j))
    return L
def genpairsnotinL(n,m, N, LL):
    L=set()
    i, j = np.random.randint(0, n), np.random.randint(0, m)
    if (i,j) not in LL:
        L.add((i,j))
    while len(L) < N:
        while (i, j) in L or (i,j) in LL:
            i, j = np.random.randint(0, n), np.random.randint(0,m)
        L.add((i,j))
    return L

def IsAutonomous(input_matrix, output_matrix):
    
    autonomous_species=True
    autonomous_reactions=True
    n, m =input_matrix.shape
    for s in range(n):
        if sum(input_matrix[s,r] for r in range(m))==0 or sum(output_matrix[s,r] for r in range(m))==0:
            autonomous_species=False
    for r in range(m):
        if sum(input_matrix[s,r] for s in range(n))==0 or sum(output_matrix[s,r] for s in range(n))==0:
            autonomous_reactions=False

    return autonomous_species, autonomous_reactions


## Data

In [5]:
# file="instancesSMGen5/5/"
# output_matrix= np.loadtxt(file+"right_side").T
# input_matrix= np.loadtxt(file+"left_side").T
#n=20
#m=20
NN=[10, 20, 30, 40, 50]
MM=[10, 20, 30, 40, 50]
UB=100
D=[3,4,5]
for n in NN:
    for m in MM:
        for d in D:
            for rep in [1,2,3,4,5]:
                matrix = np.random.randint(0,UB, size=(n,m))
                pairs1=genpairs(n, m, n*m/d)
                pairs2=genpairsnotinL(n, m, n*m/d, pairs1)

                output_matrix = np.zeros((n,m))
                input_matrix = np.zeros((n,m))
                for i in range(n):
                    for j in range(m):
                        if (i,j) in pairs1:
                            input_matrix[i,j]=matrix[i,j]
                        elif (i,j) in pairs2:
                            output_matrix[i,j]=matrix[i,j]

                np.savetxt("instances/left_n%dm%dd%d_%d.txt"%(n,m,d,rep), input_matrix, fmt="%d")
                np.savetxt("instances/right_n%dm%dd%d_%d.txt"%(n,m,d,rep), output_matrix, fmt="%d")
                #SM=output_matrix-input_matrix

## Growth factor

In [6]:
import pandas as pd
print("n \t m \t d \t rep \t steps \t time \t\t alpha \t IsAut_Sp \t IsAut_Re")
df = pd.DataFrame(columns=['n', 'm', 'd', 'rep', 'steps', 'time', 'alpha', 'x', 'IsAut_Sp', 'IsAut_Re'])
row=0
NN=[10, 20, 30, 40, 50]
MM=[10, 20, 30, 40, 50]
for n in NN:
    for m in MM:
        D=[3,4,5]
        for d in D:
            for rep in [1,2,3,4,5]:
                input_matrix= np.loadtxt("instances/left_n%dm%dd%d_%d.txt"%(n,m,d,rep))
                output_matrix= np.loadtxt("instances/right_n%dm%dd%d_%d.txt"%(n,m,d,rep))
                XX = code.growthRateGraph(output_matrix, input_matrix, 10000)
                xsol = XX[0]
                alphasol =XX[1]
                step = XX[2]
                time=XX[4]
                # print("steps: ", step)
                # print("x: ", xsol)
                # print("alpha: ", alphasol)
                # print("time: ", time)
                isA_sp, isA_re=IsAutonomous(input_matrix, output_matrix)
                print("%d \t %d \t %d \t %d \t %d \t %.4f \t %.4f"%(n,m,d,rep,step,time,alphasol), "\t", isA_sp, isA_re, sep="\t")
                df.loc[row] = [n,m,d,rep,step,time,alphasol,xsol, isA_sp, isA_re]
                row+=1
df.to_csv('GrowthRate_Experiments.csv', index=False)  

n 	 m 	 d 	 rep 	 steps 	 time 		 alpha 	 IsAut_Sp 	 IsAut_Re
10 	 10 	 3 	 1 	 32 	 0.1321 	 0.9611			True	True
10 	 10 	 3 	 2 	 51 	 0.1919 	 0.6229			True	True
10 	 10 	 3 	 3 	 44 	 0.1629 	 0.8187			True	True
10 	 10 	 3 	 4 	 42 	 0.1619 	 0.4282			True	True
10 	 10 	 3 	 5 	 0 	 0.0038 	 -1.0000			False	True
10 	 10 	 4 	 1 	 0 	 0.0041 	 0.0000			False	False
10 	 10 	 4 	 2 	 36 	 0.1389 	 0.7915			True	True
10 	 10 	 4 	 3 	 0 	 0.0036 	 -1.0000			False	False
10 	 10 	 4 	 4 	 0 	 0.0039 	 -1.0000			False	False
10 	 10 	 4 	 5 	 0 	 0.0039 	 -1.0000			False	False
10 	 10 	 5 	 1 	 0 	 0.0039 	 -1.0000			False	False
10 	 10 	 5 	 2 	 0 	 0.0039 	 -1.0000			False	False
10 	 10 	 5 	 3 	 0 	 0.0040 	 -1.0000			False	False
10 	 10 	 5 	 4 	 10001 	 36.8266 	 14.9969			True	False
10 	 10 	 5 	 5 	 0 	 0.0036 	 0.0000			False	False
10 	 20 	 3 	 1 	 55 	 0.3947 	 1.3617			True	False
10 	 20 	 3 	 2 	 47 	 0.3357 	 1.6019			True	True
10 	 20 	 3 	 3 	 70 	 0.4993 	 2.3082			True	Tru

In [13]:



XX = code.growthRateinSubgraph(output_matrix, input_matrix, 1000)
xsol = XX[0]
alphasol =XX[1]
step = XX[2]
alphadict = XX[3]
asol = XX[4]
ysol=XX[5]
zsol=XX[6]
print("steps: ", step)
print("z: ", zsol)
print("x: ", xsol[zsol])
print("y: ", ysol, "a: ", asol)
print("alpha: ", alphasol)
print("#Species: ", len(ysol), "#AutSpecies: ", len(asol), "#Reactions: ", len(zsol))
SMsol=SM[ysol,:][:,zsol]
input_matrix_sol=input_matrix[ysol,:][:,zsol]
output_matrix_sol=output_matrix[ysol,:][:,zsol]

alpha0:  0.38317757009345793
**[0] alphabar:  18872.72779748124 alpha:  0.9638224280258657
**[1] alphabar:  4909.765810891025 alpha:  1.101602858618506  AUTOCATALYTIC!
**[2] alphabar:  2541.3788881422893 alpha:  1.2188090757692494  AUTOCATALYTIC!
**[3] alphabar:  1098.2455928832267 alpha:  1.3187796661130433  AUTOCATALYTIC!
**[4] alphabar:  816.6695810561934 alpha:  2.3031318934109852  AUTOCATALYTIC!
**[5] alphabar:  388.54352506381986 alpha:  4.9224615042915065  AUTOCATALYTIC!
**[6] alphabar:  164.52333297458526 alpha:  12.295002070463324  AUTOCATALYTIC!
**[7] alphabar:  15.78140448686518 alpha:  14.33965314489562  AUTOCATALYTIC!
**[8] alphabar:  3.7286724304556174 alpha:  15.502027124914633  AUTOCATALYTIC!
**[9] alphabar:  0.47253541180793945 alpha:  15.660920907391073  AUTOCATALYTIC!
**[10] alphabar:  0.060267530461487695 alpha:  15.681388285576247  AUTOCATALYTIC!
**[11] alphabar:  0.007693081710840488 alpha:  15.684004243468673  AUTOCATALYTIC!
**[12] alphabar:  0.000982119922355195

In [14]:
import pandas as pd
SM0 = pd.DataFrame(SM)
SM0.columns = ['']*SM0.shape[1]
print(SM0.to_string(index=False))

I0 = pd.DataFrame(input_matrix_sol)
I0.columns = ['']*I0.shape[1]
print(I0.to_string(index=False))

O0 = pd.DataFrame(output_matrix_sol)
O0.columns = ['']*O0.shape[1]
print(O0.to_string(index=False))

SMsol0 = pd.DataFrame(SMsol)
SMsol0.columns = ['']*SMsol0.shape[1]
print(SMsol0.to_string(index=False))

                                                                                                                       
-13.0  13.0 -10.0   3.0 -15.0  15.0 -10.0  -9.0  15.0   0.0   0.0 -12.0   0.0  17.0  10.0 -12.0  12.0   8.0  -4.0  -3.0
  0.0   2.0 -12.0   0.0   4.0 -18.0   0.0 -14.0  10.0  -5.0  16.0  14.0  -4.0  -5.0 -18.0   0.0  18.0 -12.0 -12.0   0.0
  0.0  10.0 -13.0  -2.0   6.0   0.0   0.0   0.0   0.0   0.0  -3.0 -19.0  10.0  -4.0 -15.0   3.0   0.0   0.0  -5.0   0.0
 -2.0  -4.0   0.0   0.0  11.0   0.0   0.0  -1.0   0.0  11.0  10.0   4.0 -14.0   0.0  -8.0   8.0   0.0  10.0 -13.0   4.0
  8.0  -1.0   6.0   3.0   0.0   9.0 -13.0  18.0 -13.0  11.0   9.0   0.0  -8.0   0.0   0.0   7.0  -7.0 -17.0   0.0  -7.0
-12.0  -5.0 -17.0   5.0  19.0   0.0 -17.0   0.0   0.0   0.0   0.0 -17.0 -17.0 -11.0  10.0 -11.0   7.0   0.0   0.0   0.0
-16.0  -9.0  -6.0   0.0   0.0   2.0  -7.0  -1.0  -5.0  -1.0   0.0   0.0  19.0   6.0  -5.0   0.0  19.0   8.0   0.0   6.0
  0.0   0.0  16.0 -19.0  19.0   0.0 -11.

In [15]:
alpha_hand =  np.min([sum(output_matrix[s, r] * xsol[r] 
                         for r in zsol)
                     /
                     sum(input_matrix[s, r] * xsol[r] 
                         for r in zsol) 
                     for s in asol if sum(input_matrix[s, r] * xsol[r] 
                         for r in zsol)>0.0001])
alpha_hand

15.684386344640638